In [11]:
from bs4 import BeautifulSoup
import requests

def scrape_text_from_url(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    paragraphs = soup.find_all('p')
    return "\n".join(p.get_text() for p in paragraphs)


In [12]:
online_page = scrape_text_from_url("https://medium.com/@joulee/the-looking-glass-our-souls-need-proof-of-work-6e9318a6e885")
print(online_page)

Sign up
Sign in
Sign up
Sign in
Julie Zhuo
Follow
--
343
Listen
Share
Dear Readers: I’m starting a series exploring AI and how it might change us — the way we work and learn and relate to the world. There is so much to explore! Expect more frequency of posts going forward.
A prominent billionaire gave a graduation speech the other day. The key message? Work hard to be successful.
Another prominent billionaire gave a graduation speech a year later. The key message? Working hard is bad advice. Do what feels fun for you to be successful.
Europeans mock Americans for working too hard. Boomers lament Millenials for laziness and entitlement. Republicans fume that the work ethic America was built on is eroding like cliffs against crashing waves of handouts. Democrats believe that too much work is making us sicker and lonelier.
Today, hustling competes with YOLO. The search for spirituality wrestles with the quest for commercial success. Slow down and meditate! Hurry up and embrace AI! Technol

In [13]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')  # or any better embedding model

def embed_text(texts):
    return model.encode(texts, convert_to_tensor=False)


/Users/souvik/Documents/Codes/Github_Active/RAG_test/RAG/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [16]:
# Import the Pinecone library
from pinecone import Pinecone
key = "pcsk_4hpJ1p_GBB4PSUEzr8gfG4FybzmoYph1WNu1y9nBN668vj61wd4SaRLGfx2hdygbmQmQS4"

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key=key)

# Create a dense index with integrated embedding
index_name = "dense-index"
if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )


In [17]:
def search_pinecone(query, top_k=5):
    query_vec = embed_text([query])[0]
    results = index.query(query_vec.tolist(), top_k=top_k, include_metadata=True)
    return [match['metadata']['text'] for match in results['matches']]


In [21]:
# def ask_olama(prompt):
#     result = subprocess.run(["ollama", "run", "deepseek-r1:1.5b", prompt], capture_output=True, text=True)
#     return result.stdout


import requests

def ask_olama(prompt):
    url = "http://localhost:11434/api/generate"
    payload = {"model": "deepseek-r1:1.5b", "prompt": prompt}
    response = requests.post(url, json=payload)
    return response.json().get("response", "")


In [22]:
def rag_query(user_query):
    retrieved_docs = search_pinecone(user_query)
    context = "\n".join(retrieved_docs)
    prompt = f"Answer the question using the context below:\n\nContext:\n{context}\n\nQuestion: {user_query}"
    return ask_olama(prompt)


In [24]:
urls = ["https://medium.com/@joulee/the-looking-glass-our-souls-need-proof-of-work-6e9318a6e885"]
docs = [scrape_text_from_url(url) for url in urls]
embeddings = embed_text(docs)
# Define the function to store documents and embeddings in Pinecone
def store_in_pinecone(docs, embeddings):
	index = pc.Index(index_name)
	for i, (doc, embedding) in enumerate(zip(docs, embeddings)):
		index.upsert(vectors=[{"id": f"doc-{i}", "values": embedding.tolist(), "metadata": {"text": doc}}])

store_in_pinecone(docs, embeddings)

# Query
response = rag_query("What is the main topic of the second website?")
print(response)


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 19 Apr 2025 18:51:04 GMT', 'Content-Type': 'application/json', 'Content-Length': '103', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '408', 'x-pinecone-request-id': '795854858566672503', 'x-envoy-upstream-service-time': '173', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Vector dimension 384 does not match the dimension of the index 1024","details":[]}
